# Running germ selection on multiple processors
The code below should be put into a script and run using mpiexec.  It's primary function is to pass a MPI Comm object to `pygsti.algorithms.germselection.build_up_breadth`.

In [1]:
from __future__ import print_function
import time

import pygsti
import pygsti.construction as pc
from pygsti.construction import std2Q_XYICNOT
from pygsti.algorithms import germselection as germsel

from mpi4py import MPI
comm = MPI.COMM_WORLD

def do_greedy_germsel(gs_target, forced_germs, candidate_counts,
                      seedStart, outFilename, comm):
    #candidate_counts is a dict of keys = germ lengths, values = # of germs at that length                                                            

    tStart = time.time()

    candidate_germs = []
    for i,(germLength, count) in enumerate(candidate_counts.items()):
        if count == "all upto":
            candidate_germs.extend( pc.list_all_gatestrings_without_powers_and_cycles(
                    gs_target.gates.keys(), maxLength=germLength) )
        else:
            candidate_germs.extend( pc.list_random_gatestrings_onelen(
                    gs_target.gates.keys(), germLength, count, seed=seedStart+i))

    available_germs = pygsti.tools.remove_duplicates( forced_germs + candidate_germs )
    print("%d available germs" % len(available_germs))
    germs = germsel.build_up_breadth(gs_target, available_germs,
                     randomizationStrength=1e-3, numCopies=3, seed=1234,
                     gatePenalty=10.0, scoreFunc='all', tol=1e-6, threshold=1e5,
                     pretest=False, force=forced_germs, verbosity=5, comm=comm, memLimit=0.5*(1024**3))

    if comm is None or comm.Get_rank() == 0:
        print("Germs (%d) = \n" % len(germs), "\n".join(map(str,germs)))
        print("Total time = %gs" % (time.time()-tStart))
        pickle.dump(germs,open(outFilename,"wb"))
    return germs
                                                                                                                                         
#2Q case                                                                                                                                              
gs_target = std2Q_XYICNOT.gs_target
forced_germs = pygsti.construction.gatestring_list([(gl,) for gl in gs_target.gates.keys()]) #singletons                                                                                      
candidate_counts = { 3:"all upto", 4:30, 5:20, 6:20, 7:20, 8:20} # germLength:num_candidates                                                          
seedStart = 4
do_greedy_germsel(gs_target, forced_germs, candidate_counts,
                  seedStart, "germs_EXAMPLE.pkl", comm)

201 available germs
Starting germ set optimization. Lower score is better.
Memory estimate of 13.6 GB (0.5 GB limit) for all-Jac mode.
Memory estimate of 0.2 GB (0.5 GB limit) for single-Jac mode.
    Initial germ set computation Iter 1 of 6 Gii: 
    Initial germ set computation Iter 2 of 6 Gix: 
    Initial germ set computation Iter 3 of 6 Giy: 
    Initial germ set computation Iter 4 of 6 Gxi: 
    Initial germ set computation Iter 5 of 6 Gyi: 
    Initial germ set computation Iter 6 of 6 Gcnot: 
  Outer iteration: 1 of 1282 amplified, 6 germs
    Inner iter over candidate germs Iter 001 of 195 GiiGix: 
      Score: major=-114.0 minor=275.7550232560748, N: 194
    Inner iter over candidate germs Iter 002 of 195 GiiGiy: 
      Score: major=-114.0 minor=275.75224428270565, N: 194
    Inner iter over candidate germs Iter 003 of 195 GiiGxi: 
      Score: major=-114.0 minor=303.12985712455384, N: 194
    Inner iter over candidate germs Iter 004 of 195 GiiGyi: 
      Score: major=-114.0 m

      Score: major=-104.0 minor=360.7166056237821, N: 194
    Inner iter over candidate germs Iter 064 of 195 GixGcnotGyi: 
      Score: major=-104.0 minor=308.8900481389921, N: 194
    Inner iter over candidate germs Iter 065 of 195 GixGcnotGcnot: 
      Score: major=-104.0 minor=3769.1180162488185, N: 194
    Inner iter over candidate germs Iter 066 of 195 GiyGiyGxi: 
      Score: major=-104.0 minor=452.76438995248725, N: 194
    Inner iter over candidate germs Iter 067 of 195 GiyGiyGyi: 
      Score: major=-104.0 minor=374.9703858917059, N: 194
    Inner iter over candidate germs Iter 068 of 195 GiyGiyGcnot: 
      Score: major=-104.0 minor=331.3662851574801, N: 194
    Inner iter over candidate germs Iter 069 of 195 GiyGxiGxi: 
      Score: major=-104.0 minor=405.68196783236044, N: 194
    Inner iter over candidate germs Iter 070 of 195 GiyGxiGyi: 
      Score: major=-104.0 minor=301.8879330130469, N: 194
    Inner iter over candidate germs Iter 071 of 195 GiyGxiGcnot: 
      Score

    Inner iter over candidate germs Iter 129 of 195 GiyGiyGyiGyiGyi: 
      Score: major=-84.0 minor=341.9405848306869, N: 194
    Inner iter over candidate germs Iter 130 of 195 GiiGyiGiiGixGcnot: 
      Score: major=-84.0 minor=314.35927875108786, N: 194
    Inner iter over candidate germs Iter 131 of 195 GiyGiiGyiGiiGxi: 
      Score: major=-84.0 minor=309.3571077163643, N: 194
    Inner iter over candidate germs Iter 132 of 195 GixGiiGcnotGiiGxi: 
      Score: major=-84.0 minor=343.88500326302056, N: 194
    Inner iter over candidate germs Iter 133 of 195 GiyGxiGcnotGxiGxi: 
      Score: major=-84.0 minor=358.33465859330033, N: 194
    Inner iter over candidate germs Iter 134 of 195 GiiGyiGixGiyGii: 
      Score: major=-84.0 minor=317.4657324472904, N: 194
    Inner iter over candidate germs Iter 135 of 195 GixGcnotGxiGxiGiy: 
      Score: major=-84.0 minor=317.62490691202214, N: 194
    Inner iter over candidate germs Iter 136 of 195 GiiGyiGiyGyiGcnotGxi: 
      Score: major=-74.0

      Score: major=-54.0 minor=408.1354302733384, N: 194
    Inner iter over candidate germs Iter 191 of 195 GiiGcnotGyiGiyGiiGyiGyiGcnot: 
      Score: major=-54.0 minor=454.98502973409853, N: 194
    Inner iter over candidate germs Iter 192 of 195 GxiGcnotGixGiyGyiGyiGyiGcnot: 
      Score: major=-54.0 minor=430.81972136307354, N: 194
    Inner iter over candidate germs Iter 193 of 195 GyiGyiGcnotGxiGixGyiGixGyi: 
      Score: major=-54.0 minor=553.0107232073505, N: 194
    Inner iter over candidate germs Iter 194 of 195 GxiGiyGixGcnotGiiGyiGyiGiy: 
      Score: major=-54.0 minor=404.2006477111416, N: 194
    Inner iter over candidate germs Iter 195 of 195 GxiGixGcnotGixGxiGiyGyiGiy: 
      Score: major=-54.0 minor=404.0304308442727, N: 194
    Added GxiGyi to final germs (Score: major=-114.0 minor=257.14469808616536, N: 194)
    Added GxiGyi to final germs (Score: major=-114.0 minor=257.14469808616536, N: 194)
    Added GxiGyi to final germs (Score: major=-114.0 minor=257.1446980861

      Score: major=-110.0 minor=507.04729983705255, N: 220
    Inner iter over candidate germs Iter 060 of 194 GixGyiGcnot: 
      Score: major=-110.0 minor=433.50288926131725, N: 220
    Inner iter over candidate germs Iter 061 of 194 GixGcnotGiy: 
      Score: major=-110.0 minor=370.5530035268961, N: 220
    Inner iter over candidate germs Iter 062 of 194 GixGcnotGxi: 
      Score: major=-110.0 minor=456.3516454658403, N: 220
    Inner iter over candidate germs Iter 063 of 194 GixGcnotGyi: 
      Score: major=-110.0 minor=417.7319600630204, N: 220
    Inner iter over candidate germs Iter 064 of 194 GixGcnotGcnot: 
      Score: major=-110.0 minor=3838.4230527189966, N: 220
    Inner iter over candidate germs Iter 065 of 194 GiyGiyGxi: 
      Score: major=-110.0 minor=559.1572054523858, N: 220
    Inner iter over candidate germs Iter 066 of 194 GiyGiyGyi: 
      Score: major=-110.0 minor=454.4278306882242, N: 220
    Inner iter over candidate germs Iter 067 of 194 GiyGiyGcnot: 
      S

      Score: major=-90.0 minor=603.9217618936768, N: 220
    Inner iter over candidate germs Iter 125 of 194 GyiGixGixGiyGiy: 
      Score: major=-90.0 minor=422.18012815021166, N: 220
    Inner iter over candidate germs Iter 126 of 194 GyiGixGixGcnotGxi: 
      Score: major=-90.0 minor=568.7660762497272, N: 220
    Inner iter over candidate germs Iter 127 of 194 GcnotGcnotGcnotGixGxi: 
      Score: major=-90.0 minor=453.1879797053376, N: 220
    Inner iter over candidate germs Iter 128 of 194 GiyGiyGyiGyiGyi: 
      Score: major=-90.0 minor=421.8117103638414, N: 220
    Inner iter over candidate germs Iter 129 of 194 GiiGyiGiiGixGcnot: 
      Score: major=-90.0 minor=401.44913440855623, N: 220
    Inner iter over candidate germs Iter 130 of 194 GiyGiiGyiGiiGxi: 
      Score: major=-90.0 minor=445.2401716987465, N: 220
    Inner iter over candidate germs Iter 131 of 194 GixGiiGcnotGiiGxi: 
      Score: major=-90.0 minor=423.70376281328157, N: 220
    Inner iter over candidate germs Ite

KeyboardInterrupt: 

Above is **keyboard-interrupted on purpose**, as this output was produced with a single processor and it would have taken a very long time.
